In [1]:
#!/usr/bin/env python

import torch
import torch.nn as nn
import pickle as pk
from sklearn.model_selection import KFold
from config import config
from bert_function import *
import os


import warnings
warnings.filterwarnings('once')

words, sentences, labels, tags_vals, tag2idx, add_len = read_data(config, "data/dataset.csv") #data_path = "data/dataset.csv"
config['num_labels'] = len(tag2idx)
data_fold = vectorization(config, sentences, labels, tags_vals, tag2idx, add_len)

config['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print('The device is ' + torch.cuda.get_device_name(0)) 
print('The number of device is ', n_gpu) 





Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Using TensorFlow backend.


The longest sentence has 697 tokens.
The device is GeForce RTX 2080 Ti
The number of device is  2


In [6]:
def cross_validation1(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

    #             weight = torch.tensor(np.log(max(count)/count)+config['mu'])
        weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, precision, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [9]:
c = config.copy()
lr = []
acc = []
f1 = []
recall = []
for r in range(8,11):
    c['lr']=r*10**(-5)
    print('lr=',c['lr'])
    lr.append(c['lr'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
lr_list = list(zip(lr,acc,f1,recall))

lr= 8e-05
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9848069738480697, Macro Fbeta-Score: 0.9424542600529386, Macro Precision: 0.9013414104897292, Macro Recall: 0.9600985722002938

Fold 2:
The best result: 
Validation Accuracy: 0.9752355316285329, Macro Fbeta-Score: 0.8825289792905747, Macro Precision: 0.8667597780155134, Macro Recall: 0.9021045211345432

Fold 3:
The best result: 
Validation Accuracy: 0.9662395268605224, Macro Fbeta-Score: 0.8536087220424319, Macro Precision: 0.8693638397986223, Macro Recall: 0.8598680608352098

Fold 4:
The best result: 
Validation Accuracy: 0.969390471120575, Macro Fbeta-Score: 0.841150123599266, Macro Precision: 0.7897681147813137, Macro Recall: 0.8907313304090587

Fold 5:
The best result: 
Validation Accuracy: 0.9809670781893004, Macro Fbeta-Score: 0.9294254883181045, Macro Precision: 0.8666422411592821, Macro Recall: 0.9538696767702937
accuracy of folds: [0.9848069738480697, 0.9752355316285329, 0.9662395268605224, 0.96939047

In [10]:
lr_list_ = np.array(lr_list)
best_lr = lr_list_[np.argmax(lr_list_[:,3]),0]
print(best_lr)

9e-05


In [11]:
c = config.copy()
c['lr'] = best_lr
decay = []
acc = []
f1 = []
recall = []
for r in range(2,7):
    c['decay']=10**(-r)
    print('decay=',c['decay'])
    decay.append(c['decay'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
decay_list = list(zip(decay,acc,f1,recall))

decay= 0.01
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9748896247240618, Macro Fbeta-Score: 0.9191993666769112, Macro Precision: 0.8200195965366568, Macro Recall: 0.9632005042551086

Fold 2:
The best result: 
Validation Accuracy: 0.9691281797974809, Macro Fbeta-Score: 0.8959741652649555, Macro Precision: 0.861778507045708, Macro Recall: 0.9090544379258891

Fold 3:
The best result: 
Validation Accuracy: 0.9741614906832298, Macro Fbeta-Score: 0.8453370715276434, Macro Precision: 0.8954332402219756, Macro Recall: 0.8388772257275805

Fold 4:
The best result: 
Validation Accuracy: 0.9835657370517928, Macro Fbeta-Score: 0.9225503232212614, Macro Precision: 0.8865771308245562, Macro Recall: 0.9510723020712037

Fold 5:
The best result: 
Validation Accuracy: 0.9725732723850498, Macro Fbeta-Score: 0.9044580536921887, Macro Precision: 0.7913361501982822, Macro Recall: 0.9599115617838676
accuracy of folds: [0.9748896247240618, 0.9691281797974809, 0.9741614906832298, 0.98356

In [12]:
decay_list_ = np.array(decay_list)
best_decay = decay_list_[np.argmax(decay_list_[:,3]),0]
print(best_decay)

0.001


In [8]:
decay_list_

array([[1.00000000e-01, 9.77625457e-01, 9.07463190e-01, 9.26710955e-01],
       [1.00000000e-02, 9.77573887e-01, 8.82764944e-01, 9.02951218e-01],
       [1.00000000e-03, 9.76280130e-01, 9.09387706e-01, 9.31017738e-01],
       [1.00000000e-04, 9.73050954e-01, 8.80538252e-01, 8.97310409e-01],
       [1.00000000e-05, 9.78525678e-01, 9.21663665e-01, 9.37345464e-01],
       [1.00000000e-06, 9.77870271e-01, 9.11267617e-01, 9.29764452e-01]])

In [2]:
def cross_validation1(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

    #             weight = torch.tensor(np.log(max(count)/count)+config['mu'])
        weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, precision, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [5]:
c = config.copy()
lr = []
acc = []
f1 = []
recall = []
for r in range(5,11):
    c['lr']=r*10**(-5)
    print('lr=',c['lr'])
    lr.append(c['lr'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
lr_list = list(zip(lr,acc,f1,recall))

lr= 5e-05
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9812920927912198, Macro Fbeta-Score: 0.9341531996352135, Macro Precision: 0.893959377578103, Macro Recall: 0.9527198854807375

Fold 2:
The best result: 
Validation Accuracy: 0.9865071283095723, Macro Fbeta-Score: 0.8764877358199594, Macro Precision: 0.8783789925957629, Macro Recall: 0.8769526396623503

Fold 3:
The best result: 
Validation Accuracy: 0.9811503141614306, Macro Fbeta-Score: 0.8677767686933748, Macro Precision: 0.8131276825583726, Macro Recall: 0.8850211739667112

Fold 4:
The best result: 
Validation Accuracy: 0.9768266085059978, Macro Fbeta-Score: 0.8856435271223334, Macro Precision: 0.8901496445836817, Macro Recall: 0.8874348545954788

Fold 5:
The best result: 
Validation Accuracy: 0.9758807588075881, Macro Fbeta-Score: 0.9256524400057773, Macro Precision: 0.8901735917417744, Macro Recall: 0.9424331633517103
accuracy of folds: [0.9812920927912198, 0.9865071283095723, 0.9811503141614306, 0.9768266

In [6]:
lr_list_ = np.array(lr_list)
best_lr = lr_list_[np.argmax(lr_list_[:,3]),0]
print(best_lr)

8e-05


In [7]:
c = config.copy()
c['lr'] = best_lr
decay = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['decay']=10**(-r)
    print('decay=',c['decay'])
    decay.append(c['decay'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
decay_list = list(zip(decay,acc,f1,recall))

decay= 0.1
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9903105590062112, Macro Fbeta-Score: 0.9679015299043431, Macro Precision: 0.9402000002947445, Macro Recall: 0.9763538166395528

Fold 2:
The best result: 
Validation Accuracy: 0.968031968031968, Macro Fbeta-Score: 0.8596225321068818, Macro Precision: 0.8486031299957817, Macro Recall: 0.8760214289273006

Fold 3:
The best result: 
Validation Accuracy: 0.9809088464641033, Macro Fbeta-Score: 0.9123538597433691, Macro Precision: 0.8627574204828522, Macro Recall: 0.9293304596259643

Fold 4:
The best result: 
Validation Accuracy: 0.9807351077313055, Macro Fbeta-Score: 0.9336823180262536, Macro Precision: 0.9019637288666791, Macro Recall: 0.9439682793042405

Fold 5:
The best result: 
Validation Accuracy: 0.986096256684492, Macro Fbeta-Score: 0.8711130775037225, Macro Precision: 0.8630811845680871, Macro Recall: 0.8751653264581561
accuracy of folds: [0.9903105590062112, 0.968031968031968, 0.9809088464641033, 0.98073510

The best result: 
Validation Accuracy: 0.9757958431991581, Macro Fbeta-Score: 0.8688238786243067, Macro Precision: 0.8404131021756993, Macro Recall: 0.8900008735011785

Fold 2:
The best result: 
Validation Accuracy: 0.9885790172642762, Macro Fbeta-Score: 0.9312121972310875, Macro Precision: 0.9469987596075568, Macro Recall: 0.9312591105195863

Fold 3:
The best result: 
Validation Accuracy: 0.9738666666666667, Macro Fbeta-Score: 0.9169378509860172, Macro Precision: 0.8532277934267858, Macro Recall: 0.9477124615633085

Fold 4:
The best result: 
Validation Accuracy: 0.9758754863813229, Macro Fbeta-Score: 0.8784451147813452, Macro Precision: 0.8842031068047707, Macro Recall: 0.8787884924810686

Fold 5:
The best result: 
Validation Accuracy: 0.9781792585640544, Macro Fbeta-Score: 0.9121660743348113, Macro Precision: 0.8506894234964265, Macro Recall: 0.939399318214467
accuracy of folds: [0.9757958431991581, 0.9885790172642762, 0.9738666666666667, 0.9758754863813229, 0.9781792585640544]
avera

In [8]:
decay_list_ = np.array(decay_list)
best_decay = decay_list_[np.argmax(decay_list_[:,3]),0]
print(best_decay)

0.001


In [3]:
def cross_validation2(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.log(max(count)/count)+config['mu'])
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, _, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [4]:
c = config.copy()
best_lr = c['lr']
best_decay = c['decay']

In [5]:
c = config.copy()
c['lr'] = best_lr
c['decay'] = best_decay
mu = []
acc = []
f1 = []
recall = []
for r in range(5,105,5):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 0.5
Cross Validation

Fold 1:


/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The best result: 
Validation Accuracy: 0.9792261750194755, Macro Fbeta-Score: 0.8279434474743719, Macro Precision: 0.7857707565532469, Macro Recall: 0.8408885959260013

Fold 2:
The best result: 
Validation Accuracy: 0.9694817153380689, Macro Fbeta-Score: 0.8737398710890645, Macro Precision: 0.8205658491569813, Macro Recall: 0.8913207177979573

Fold 3:
The best result: 
Validation Accuracy: 0.9721227621483376, Macro Fbeta-Score: 0.9148198698686101, Macro Precision: 0.8375695628453207, Macro Recall: 0.9526730746268897

Fold 4:
The best result: 
Validation Accuracy: 0.9798376538360828, Macro Fbeta-Score: 0.9099681744200645, Macro Precision: 0.8733258230852253, Macro Recall: 0.9253809383408227

Fold 5:
The best result: 
Validation Accuracy: 0.9827245804540967, Macro Fbeta-Score: 0.9260606031676237, Macro Precision: 0.8806279657879638, Macro Recall: 0.9409292497891799
accuracy of folds: [0.9792261750194755, 0.9694817153380689, 0.9721227621483376, 0.9798376538360828, 0.9827245804540967]
aver

The best result: 
Validation Accuracy: 0.9790090309982914, Macro Fbeta-Score: 0.8896288231091558, Macro Precision: 0.8641607385422969, Macro Recall: 0.9011009600616544

Fold 3:
The best result: 
Validation Accuracy: 0.9589654314847053, Macro Fbeta-Score: 0.847010557573969, Macro Precision: 0.7391235858763133, Macro Recall: 0.8976618774422642

Fold 4:
The best result: 
Validation Accuracy: 0.9778147531891292, Macro Fbeta-Score: 0.9178058146782758, Macro Precision: 0.8774642254035443, Macro Recall: 0.9384788447640765

Fold 5:
The best result: 
Validation Accuracy: 0.982673907421774, Macro Fbeta-Score: 0.881633482798308, Macro Precision: 0.8534477341549647, Macro Recall: 0.8912333920357589
accuracy of folds: [0.9789172306090578, 0.9790090309982914, 0.9589654314847053, 0.9778147531891292, 0.982673907421774]
average accuracy: 0.9754760707405916
f1 of folds: [0.9151847756346213, 0.8896288231091558, 0.847010557573969, 0.9178058146782758, 0.881633482798308]
average f1: 0.890252690758866
recall

The best result: 
Validation Accuracy: 0.9815983175604627, Macro Fbeta-Score: 0.9049349978382798, Macro Precision: 0.8627472139614701, Macro Recall: 0.9276225828453357

Fold 4:
The best result: 
Validation Accuracy: 0.9717028802425467, Macro Fbeta-Score: 0.8785661525072376, Macro Precision: 0.8337911489013288, Macro Recall: 0.9065465839123727

Fold 5:
The best result: 
Validation Accuracy: 0.9822528962287405, Macro Fbeta-Score: 0.8897975687612635, Macro Precision: 0.8542332972373772, Macro Recall: 0.9031254085203796
accuracy of folds: [0.976658798846053, 0.9792159957905814, 0.9815983175604627, 0.9717028802425467, 0.9822528962287405]
average accuracy: 0.9782857777336769
f1 of folds: [0.9450233501371895, 0.8804895552072032, 0.9049349978382798, 0.8785661525072376, 0.8897975687612635]
average f1: 0.8997623248902347
recall of folds: [0.9542749515010941, 0.8980234240773324, 0.9276225828453357, 0.9065465839123727, 0.9031254085203796]
average recall: 0.917918590171303
mu= 7.0
Cross Validation


The best result: 
Validation Accuracy: 0.9729866195405201, Macro Fbeta-Score: 0.8357883595674607, Macro Precision: 0.8038212799842057, Macro Recall: 0.8487186342704389

Fold 5:
The best result: 
Validation Accuracy: 0.9766796224319823, Macro Fbeta-Score: 0.9146318008042352, Macro Precision: 0.8546825151963647, Macro Recall: 0.9391638179251954
accuracy of folds: [0.9683527885862516, 0.9797877716018191, 0.9770766576287897, 0.9729866195405201, 0.9766796224319823]
average accuracy: 0.9749766919578725
f1 of folds: [0.9070885945382314, 0.9058201366256378, 0.8643581825139421, 0.8357883595674607, 0.9146318008042352]
average f1: 0.8855374148099013
recall of folds: [0.9297714018134224, 0.9123308709925784, 0.8887045738455259, 0.8487186342704389, 0.9391638179251954]
average recall: 0.9037378597694323
mu= 10.0
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9717827004219409, Macro Fbeta-Score: 0.8922137579814649, Macro Precision: 0.848408958491565, Macro Recall: 0.906905338110249

In [6]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

1.5


# test without loss

In [14]:
def cross_validation2(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.log(max(count)/count)+config['mu'])
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, _, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [27]:
def BuildModel(config, weight=None):
    # change the forward method: do not consider 'X' when computing loss
    def new_forward(self, input_ids, token_type_ids=None, attention_mask=None, add_masks=None, labels=None, weight=weight):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            if weight is not None:
                weight = weight.to(torch.float).to(config['device'])
            loss_fct = nn.CrossEntropyLoss(weight=weight, ignore_index=self.num_labels-1)
            # Only keep active parts of the loss
            if attention_mask is not None or add_masks is not None:
                if add_masks is None:
                    add_masks = 1
                if attention_mask is None:
                    attention_mask = 1
                active_loss = (attention_mask.view(-1) == 1) * (add_masks.view(-1) == 1) 
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits
    BertForTokenClassification.forward = new_forward
    model = BertForTokenClassification.from_pretrained(config['name'], num_labels=config['num_labels'])
#     if torch.cuda.device_count() > 1:
#         model = nn.DataParallel(model)
    model.to(config['device'])
    
    return model

In [28]:
c = config.copy()
mu = []
acc = []
f1 = []
recall = []
for r in range(10,21):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 1.0
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9767790262172285, Macro Fbeta-Score: 0.9222741763601849, Macro Precision: 0.887213685669146, Macro Recall: 0.936027726182879

Fold 2:
The best result: 
Validation Accuracy: 0.9774785801713586, Macro Fbeta-Score: 0.9595361454726906, Macro Precision: 0.9311617845613794, Macro Recall: 0.9679363050952978

Fold 3:
The best result: 
Validation Accuracy: 0.979493670886076, Macro Fbeta-Score: 0.9044412161740362, Macro Precision: 0.8187894715501826, Macro Recall: 0.9414204168914047

Fold 4:
The best result: 
Validation Accuracy: 0.9704990350151641, Macro Fbeta-Score: 0.8651416563140053, Macro Precision: 0.7985101183306287, Macro Recall: 0.898151699335515

Fold 5:
The best result: 
Validation Accuracy: 0.9763674986723314, Macro Fbeta-Score: 0.927470206090754, Macro Precision: 0.881081331131758, Macro Recall: 0.9435149163030823
accuracy of folds: [0.9767790262172285, 0.9774785801713586, 0.979493670886076, 0.970499035015164

The best result: 
Validation Accuracy: 0.9824849480021893, Macro Fbeta-Score: 0.9618176119770409, Macro Precision: 0.8908367405665941, Macro Recall: 0.9881535465353749

Fold 2:
The best result: 
Validation Accuracy: 0.9744562468386444, Macro Fbeta-Score: 0.8870901161689275, Macro Precision: 0.8102323864235211, Macro Recall: 0.9247804121782727

Fold 3:
The best result: 
Validation Accuracy: 0.9704694598687531, Macro Fbeta-Score: 0.8114097940483683, Macro Precision: 0.7881137874239419, Macro Recall: 0.8431209493052622

Fold 4:
The best result: 
Validation Accuracy: 0.9737980157720681, Macro Fbeta-Score: 0.9404302218196037, Macro Precision: 0.8922634477347839, Macro Recall: 0.9543417762700984

Fold 5:
The best result: 
Validation Accuracy: 0.9883011190233978, Macro Fbeta-Score: 0.9617231592823653, Macro Precision: 0.9586498519848198, Macro Recall: 0.9633521468216322
accuracy of folds: [0.9824849480021893, 0.9744562468386444, 0.9704694598687531, 0.9737980157720681, 0.9883011190233978]
aver

# with loss

In [25]:
def BuildModel(config, weight=None):
    # change the forward method: do not consider 'X' when computing loss
    def new_forward(self, input_ids, token_type_ids=None, attention_mask=None, add_masks=None, labels=None, weight=weight):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            if weight is not None:
                weight = weight.to(torch.float).to(config['device'])
            loss_fct = nn.CrossEntropyLoss(weight=weight, ignore_index=self.num_labels-1)
            # Only keep active parts of the loss
            if attention_mask is not None or add_masks is not None:
                if add_masks is None:
                    add_masks = 1
                if attention_mask is None:
                    attention_mask = 1
                active_loss = (attention_mask.view(-1) == 1)# * (add_masks.view(-1) == 1) 
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits
    BertForTokenClassification.forward = new_forward
    model = BertForTokenClassification.from_pretrained(config['name'], num_labels=config['num_labels'])
#     if torch.cuda.device_count() > 1:
#         model = nn.DataParallel(model)
    model.to(config['device'])
    
    return model

In [26]:
c = config.copy()
mu = []
acc = []
f1 = []
recall = []
for r in range(10,21):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 1.0
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.985686219477968, Macro Fbeta-Score: 0.9440792649961003, Macro Precision: 0.8865496756113629, Macro Recall: 0.9721623993274021

Fold 2:
The best result: 
Validation Accuracy: 0.9555719194894453, Macro Fbeta-Score: 0.8588022433772884, Macro Precision: 0.8201009448452911, Macro Recall: 0.8757480993570567

Fold 3:
The best result: 
Validation Accuracy: 0.9775853285787061, Macro Fbeta-Score: 0.9217640990660011, Macro Precision: 0.8458097974237773, Macro Recall: 0.9768362092171455

Fold 4:
The best result: 
Validation Accuracy: 0.9804890738813735, Macro Fbeta-Score: 0.9439830725175026, Macro Precision: 0.8607922743728899, Macro Recall: 0.9765363051616661

Fold 5:
The best result: 
Validation Accuracy: 0.9783904619970194, Macro Fbeta-Score: 0.8731720547618376, Macro Precision: 0.8129179080921113, Macro Recall: 0.9062195794775627
accuracy of folds: [0.985686219477968, 0.9555719194894453, 0.9775853285787061, 0.9804890738

The best result: 
Validation Accuracy: 0.9766807243860084, Macro Fbeta-Score: 0.9240129394075226, Macro Precision: 0.8858392919287001, Macro Recall: 0.937572778427455

Fold 2:
The best result: 
Validation Accuracy: 0.9748818701815469, Macro Fbeta-Score: 0.9061800036338753, Macro Precision: 0.800052458970306, Macro Recall: 0.9610581314465191

Fold 3:
The best result: 
Validation Accuracy: 0.9825903455552625, Macro Fbeta-Score: 0.9100445932602628, Macro Precision: 0.8903372820100791, Macro Recall: 0.9185123674096543

Fold 4:
The best result: 
Validation Accuracy: 0.9789851358277806, Macro Fbeta-Score: 0.9312498214242056, Macro Precision: 0.9215065385888637, Macro Recall: 0.9359874033738186

Fold 5:
The best result: 
Validation Accuracy: 0.9804772234273319, Macro Fbeta-Score: 0.9278330980887834, Macro Precision: 0.8783575750521421, Macro Recall: 0.9498978887899256
accuracy of folds: [0.9766807243860084, 0.9748818701815469, 0.9825903455552625, 0.9789851358277806, 0.9804772234273319]
averag

In [29]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

1.4


# no weight with loss

In [7]:
c = config.copy()
acc = []
f1 = []
recall = []
ave_acc, ave_f1, ave_recall = cross_validation1(c)
acc.append(ave_acc)
f1.append(ave_f1)
recall.append(ave_recall)

Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9846115661454455, Macro Fbeta-Score: 0.9510020218539653, Macro Precision: 0.8732934877642747, Macro Recall: 0.9755810701340419

Fold 2:
The best result: 
Validation Accuracy: 0.9851770518803185, Macro Fbeta-Score: 0.9201627202960031, Macro Precision: 0.9564465968430877, Macro Recall: 0.9163275632160724

Fold 3:
The best result: 
Validation Accuracy: 0.9711975745325923, Macro Fbeta-Score: 0.874884747511024, Macro Precision: 0.8222551882410067, Macro Recall: 0.8946188002700647

Fold 4:
The best result: 
Validation Accuracy: 0.9732844421162913, Macro Fbeta-Score: 0.9064559326230318, Macro Precision: 0.843870085862712, Macro Recall: 0.9366373012818557

Fold 5:


/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The best result: 
Validation Accuracy: 0.9744040150564617, Macro Fbeta-Score: 0.8200007459005415, Macro Precision: 0.8184650433724143, Macro Recall: 0.8323394480148919
accuracy of folds: [0.9846115661454455, 0.9851770518803185, 0.9711975745325923, 0.9732844421162913, 0.9744040150564617]
average accuracy: 0.9777349299462219
f1 of folds: [0.9510020218539653, 0.9201627202960031, 0.874884747511024, 0.9064559326230318, 0.8200007459005415]
average f1: 0.8945012336369131
recall of folds: [0.9755810701340419, 0.9163275632160724, 0.8946188002700647, 0.9366373012818557, 0.8323394480148919]
average recall: 0.9111008365833854


# no weight without loss

In [8]:
def BuildModel(config, weight=None):
    # change the forward method: do not consider 'X' when computing loss
    def new_forward(self, input_ids, token_type_ids=None, attention_mask=None, add_masks=None, labels=None, weight=weight):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            if weight is not None:
                weight = weight.to(torch.float).to(config['device'])
            loss_fct = nn.CrossEntropyLoss(weight=weight, ignore_index=self.num_labels-1)
            # Only keep active parts of the loss
            if attention_mask is not None or add_masks is not None:
                if add_masks is None:
                    add_masks = 1
                if attention_mask is None:
                    attention_mask = 1
                active_loss = (attention_mask.view(-1) == 1) * (add_masks.view(-1) == 1) 
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits
    BertForTokenClassification.forward = new_forward
    model = BertForTokenClassification.from_pretrained(config['name'], num_labels=config['num_labels'])
#     if torch.cuda.device_count() > 1:
#         model = nn.DataParallel(model)
    model.to(config['device'])
    
    return model

In [9]:
c = config.copy()
acc = []
f1 = []
recall = []
ave_acc, ave_f1, ave_recall = cross_validation1(c)
acc.append(ave_acc)
f1.append(ave_f1)
recall.append(ave_recall)

Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.973989898989899, Macro Fbeta-Score: 0.8295906939610335, Macro Precision: 0.862246761049647, Macro Recall: 0.8274024956519825

Fold 2:
The best result: 
Validation Accuracy: 0.9714544630720435, Macro Fbeta-Score: 0.9259651333176854, Macro Precision: 0.8604254349802456, Macro Recall: 0.9479979233441586

Fold 3:
The best result: 
Validation Accuracy: 0.9875286332400102, Macro Fbeta-Score: 0.9718665296183389, Macro Precision: 0.9512432083111088, Macro Recall: 0.9776368313716817

Fold 4:
The best result: 
Validation Accuracy: 0.9835918854415274, Macro Fbeta-Score: 0.8949536539082867, Macro Precision: 0.8443465805164319, Macro Recall: 0.9214189278932593

Fold 5:
The best result: 
Validation Accuracy: 0.9788247750132345, Macro Fbeta-Score: 0.9100859876057071, Macro Precision: 0.8645844501705657, Macro Recall: 0.9355542281684506
accuracy of folds: [0.973989898989899, 0.9714544630720435, 0.9875286332400102, 0.9835918854415274, 0

In [2]:
def cross_validation3(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.maximun(np.log(config['mu']*sum(count)/count),1))
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1,_, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall

In [3]:
c = config.copy()
c['lr'] = best_lr
c['decay'] = best_decay
mu = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

NameError: name 'best_lr' is not defined

In [ ]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

In [2]:
def cross_validation3(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.power(sum(count)/count, config['mu']))
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, _, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall

In [4]:
# c['lr'] = best_lr
# c['decay'] = best_decay
c = config.copy()
mu = []
acc = []
f1 = []
recall = []
for r in range(7,21):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation3(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 0.7
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.971928849360756, Macro Fbeta-Score: 0.898095125434102, Macro Precision: 0.8143961282845201, Macro Recall: 0.9349919831131592

Fold 2:
The best result: 
Validation Accuracy: 0.9293144838623633, Macro Fbeta-Score: 0.8398387862766967, Macro Precision: 0.6937663629495332, Macro Recall: 0.940664370771665

Fold 3:
The best result: 
Validation Accuracy: 0.970131702728128, Macro Fbeta-Score: 0.9350423841950124, Macro Precision: 0.8743589365293222, Macro Recall: 0.9529141924704702

Fold 4:
The best result: 
Validation Accuracy: 0.9704228794915668, Macro Fbeta-Score: 0.8568137295015021, Macro Precision: 0.7908277557392661, Macro Recall: 0.9004574247689939

Fold 5:
The best result: 
Validation Accuracy: 0.9786267699706118, Macro Fbeta-Score: 0.9468642752850462, Macro Precision: 0.8822556743151178, Macro Recall: 0.9789829904210907
accuracy of folds: [0.971928849360756, 0.9293144838623633, 0.970131702728128, 0.97042287949156

The best result: 
Validation Accuracy: 0.9300349825087456, Macro Fbeta-Score: 0.8482035366690162, Macro Precision: 0.6875241678312562, Macro Recall: 0.9482396709570622

Fold 3:
The best result: 
Validation Accuracy: 0.8503248375812094, Macro Fbeta-Score: 0.8492620694776554, Macro Precision: 0.66811481564933, Macro Recall: 0.9654108258362037

Fold 4:
The best result: 
Validation Accuracy: 0.9303830227743272, Macro Fbeta-Score: 0.8389409714318532, Macro Precision: 0.666020408959339, Macro Recall: 0.9207399881648465

Fold 5:
The best result: 
Validation Accuracy: 0.9014750904536599, Macro Fbeta-Score: 0.8247657145736913, Macro Precision: 0.6098812658254247, Macro Recall: 0.9785067282601955
accuracy of folds: [0.9086102719033232, 0.9300349825087456, 0.8503248375812094, 0.9303830227743272, 0.9014750904536599]
average accuracy: 0.9041656410442531
f1 of folds: [0.805367893025028, 0.8482035366690162, 0.8492620694776554, 0.8389409714318532, 0.8247657145736913]
average f1: 0.8333080370354488
rec

The best result: 
Validation Accuracy: 0.5777310924369747, Macro Fbeta-Score: 0.5853917125174677, Macro Precision: 0.35426794944155543, Macro Recall: 0.8613599223884656

Fold 4:
The best result: 
Validation Accuracy: 0.6740442655935613, Macro Fbeta-Score: 0.6588981282293341, Macro Precision: 0.4530362856474622, Macro Recall: 0.923495140604464

Fold 5:
The best result: 
Validation Accuracy: 0.723267060720043, Macro Fbeta-Score: 0.6884145385516304, Macro Precision: 0.4263053237342334, Macro Recall: 0.9607567226610094
accuracy of folds: [0.6480225988700565, 0.45793914599846586, 0.5777310924369747, 0.6740442655935613, 0.723267060720043]
average accuracy: 0.6162008327238203
f1 of folds: [0.6544673919153078, 0.6611016684170589, 0.5853917125174677, 0.6588981282293341, 0.6884145385516304]
average f1: 0.6496546879261598
recall of folds: [0.9046824851566133, 0.8412045379825032, 0.8613599223884656, 0.923495140604464, 0.9607567226610094]
average recall: 0.8982997617586111
mu= 2.0
Cross Validation


In [5]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

1.0
